In [ ]:
import torch
import time
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# GPU Specs (RTX 3060)
# -----------------------------
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
gpu_name = torch.cuda.get_device_properties(device).name
peak_flops = 13.1e12        # 13.1 TFLOPs FP32
memory_bandwidth = 360e9    # 360 GB/s

# -----------------------------
# Kernel runner with timing
# -----------------------------
def run_kernel(flops, bytes_transferred, kernel_fn, warmup_iters=3, test_iters=5):
    for _ in range(warmup_iters):
        kernel_fn()
    torch.cuda.synchronize()
    start = time.time()
    for _ in range(test_iters):
        kernel_fn()
    torch.cuda.synchronize()
    end = time.time()
    runtime = (end - start) / test_iters
    achieved_flops = flops / runtime
    achieved_bw = bytes_transferred / runtime
    arithmetic_intensity = flops / bytes_transferred
    return arithmetic_intensity, achieved_flops, achieved_bw, runtime

# -----------------------------
# Sweep a range of N values
# -----------------------------
N_values = [2**n for n in range(6, 15)]

ais = []
flops_list = []
bw_list = []
runtime_list = []

for N in N_values:
    A = torch.rand((N, N), device=device)
    B = torch.rand((N, N), device=device)
    C = torch.empty((N, N), device=device)
    flops = 2 * N**3
    bytes_transferred = 3 * (N**2 * 4)

    def matmul():
        C[:] = torch.matmul(A, B)

    ai, af, abw, rt = run_kernel(flops, bytes_transferred, matmul)
    ais.append(ai)
    flops_list.append(af)
    bw_list.append(abw)
    runtime_list.append(rt)

# -----------------------------
# Create 4x1 plots
# -----------------------------
fig, axes = plt.subplots(4, figsize=(12,20))
plt.subplots_adjust(hspace=0.3, wspace=0.3)

# Roofline
ax_roof = axes[0]
ai_range = np.logspace(-2, 3.5, 500)
perf_memory_bound = ai_range * memory_bandwidth
perf_compute_bound = np.full_like(ai_range, peak_flops)
roofline = np.minimum(perf_memory_bound, perf_compute_bound)
ax_roof.loglog(ai_range, roofline/1e12, label="Roofline", color='gray', linewidth=2)
ax_roof.scatter(ais, np.array(flops_list)/1e12, color='red', s=50, label="Matrix Multiply")
ax_roof.set_xlabel("Arithmetic Intensity [FLOPs/Byte]")
ax_roof.set_ylabel("Performance [TFLOPs]")
ax_roof.set_title(f"Roofline - {gpu_name}")
ax_roof.grid(True, which="both", ls="--", alpha=0.5)
ax_roof.legend()

# Achieved FLOPs vs N
ax_flops = axes[1]
ax_flops.plot(N_values, np.array(flops_list)/1e12, '-o', color='blue')
ax_flops.set_xlabel("Matrix Size N")
ax_flops.set_ylabel("Achieved FLOPs [TFLOPs]")
ax_flops.set_title("Achieved FLOPs vs N")
ax_flops.grid(True, ls="--", alpha=0.5)

# Memory bandwidth vs N
ax_bw = axes[2]
ax_bw.plot(N_values, np.array(bw_list)/1e9, '-o', color='green')
ax_bw.set_xlabel("Matrix Size N")
ax_bw.set_ylabel("Achieved Memory Bandwidth [GB/s]")
ax_bw.set_title("Memory Bandwidth vs N")
ax_bw.grid(True, ls="--", alpha=0.5)

# Runtime vs N
ax_rt = axes[3]
ax_rt.plot(N_values, runtime_list, '-o', color='purple')
ax_rt.set_xlabel("Matrix Size N")
ax_rt.set_ylabel("Runtime [s]")
ax_rt.set_title("Kernel Runtime vs N")
ax_rt.grid(True, ls="--", alpha=0.5)

plt.show()

# Observation
# 1. When N is small the memory bandwidth is far from peak, therefore the perf is far from the roofline
# 2. Memory ~ O(N^2) while FLOP ~ O(N^3); therefore as N increase, the percentage of time spent on memory access drops, i.e., lower memory bandwidth
